<a href="https://colab.research.google.com/github/Joovvhan/korean-stt/blob/master/kang/num.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import scipy as sp
import torch